<a href="https://colab.research.google.com/github/Dharani-12b3/Air-Pollution-Prediction/blob/main/AirPollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install -q gradio

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
#importing libraries
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from tabulate import tabulate


In [57]:
train_data = pd.read_csv('/content/drive/MyDrive/machine learning project/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/machine learning project/test .csv')
op_data = pd.read_csv('/content/drive/MyDrive/machine learning project/sample_submission.csv')


In [58]:
train_data.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [59]:
test_data.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [60]:
op_data.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,2.0,10.0,300.0
1,2011-01-01 01:00:00,2.0,10.0,300.0
2,2011-01-01 02:00:00,2.0,10.0,300.0
3,2011-01-01 03:00:00,2.0,10.0,300.0
4,2011-01-01 04:00:00,2.0,10.0,300.0


In [61]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date_time               7111 non-null   object 
 1   deg_C                   7111 non-null   float64
 2   relative_humidity       7111 non-null   float64
 3   absolute_humidity       7111 non-null   float64
 4   sensor_1                7111 non-null   float64
 5   sensor_2                7111 non-null   float64
 6   sensor_3                7111 non-null   float64
 7   sensor_4                7111 non-null   float64
 8   sensor_5                7111 non-null   float64
 9   target_carbon_monoxide  7111 non-null   float64
 10  target_benzene          7111 non-null   float64
 11  target_nitrogen_oxides  7111 non-null   float64
dtypes: float64(11), object(1)
memory usage: 666.8+ KB


In [62]:

features = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
target = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']


In [63]:
X_train, X_val, y_train, y_val = train_test_split(train_data[features], train_data[target], test_size=0.2, random_state=42)


**Linear Regression**

In [64]:
'''from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

# Initialize LinearRegression models
linear_reg_carbon_monoxide = LinearRegression()
linear_reg_benzene = LinearRegression()
linear_reg_nitrogen_oxides = LinearRegression()

# Fit LinearRegression models
linear_reg_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])
linear_reg_benzene.fit(X_train, y_train['target_benzene'])
linear_reg_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])

# Predict using LinearRegression models
carbon_monoxide_preds = linear_reg_carbon_monoxide.predict(X_val)
benzene_preds = linear_reg_benzene.predict(X_val)
nitrogen_oxides_preds = linear_reg_nitrogen_oxides.predict(X_val)

from sklearn.metrics import mean_squared_error

# Calculate RMSE instead of RMSLE
rmse_carbon_monoxide = np.sqrt(mean_squared_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds))
rmse_benzene = np.sqrt(mean_squared_error(y_val['target_benzene'], benzene_preds))
rmse_nitrogen_oxides = np.sqrt(mean_squared_error(y_val['target_nitrogen_oxides'], nitrogen_oxides_preds))

# Print RMSE values
print('RMSE for carbon monoxide:', rmse_carbon_monoxide)
print('RMSE for benzene:', rmse_benzene)
print('RMSE for nitrogen oxides:', rmse_nitrogen_oxides)

RMSE for carbon monoxide: 0.6165333056387582
RMSE for benzene: 1.838907674366134
RMSE for nitrogen oxides: 113.65390626798047
'''

"from sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_log_error\n\n# Initialize LinearRegression models\nlinear_reg_carbon_monoxide = LinearRegression()\nlinear_reg_benzene = LinearRegression()\nlinear_reg_nitrogen_oxides = LinearRegression()\n\n# Fit LinearRegression models\nlinear_reg_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])\nlinear_reg_benzene.fit(X_train, y_train['target_benzene'])\nlinear_reg_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])\n\n# Predict using LinearRegression models\ncarbon_monoxide_preds = linear_reg_carbon_monoxide.predict(X_val)\nbenzene_preds = linear_reg_benzene.predict(X_val)\nnitrogen_oxides_preds = linear_reg_nitrogen_oxides.predict(X_val)\n\nfrom sklearn.metrics import mean_squared_error\n\n# Calculate RMSE instead of RMSLE\nrmse_carbon_monoxide = np.sqrt(mean_squared_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds))\nrmse_benzene = np.sqrt(mean_squared_error(y_val['

 **Random Forest Regression**

In [65]:
'''rf_carbon_monoxide = RandomForestRegressor(n_estimators=100, random_state=42)
rf_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])
carbon_monoxide_preds = rf_carbon_monoxide.predict(X_val)

rf_benzene = RandomForestRegressor(n_estimators=100, random_state=42)
rf_benzene.fit(X_train, y_train['target_benzene'])
benzene_preds = rf_benzene.predict(X_val)

rf_nitrogen_oxides = RandomForestRegressor(n_estimators=100, random_state=42)
rf_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])
nitrogen_oxides_preds = rf_nitrogen_oxides.predict(X_val)
'''

"rf_carbon_monoxide = RandomForestRegressor(n_estimators=100, random_state=42)\nrf_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])\ncarbon_monoxide_preds = rf_carbon_monoxide.predict(X_val)\n\nrf_benzene = RandomForestRegressor(n_estimators=100, random_state=42)\nrf_benzene.fit(X_train, y_train['target_benzene'])\nbenzene_preds = rf_benzene.predict(X_val)\n\nrf_nitrogen_oxides = RandomForestRegressor(n_estimators=100, random_state=42)\nrf_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])\nnitrogen_oxides_preds = rf_nitrogen_oxides.predict(X_val)\n"

 **Comparision of model**

In [66]:
linear_reg_carbon_monoxide = LinearRegression()
linear_reg_benzene = LinearRegression()
linear_reg_nitrogen_oxides = LinearRegression()
# Fit LinearRegression models
linear_reg_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])
linear_reg_benzene.fit(X_train, y_train['target_benzene'])
linear_reg_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])

# Predict using LinearRegression models
carbon_monoxide_preds_lr = linear_reg_carbon_monoxide.predict(X_val)
benzene_preds_lr = linear_reg_benzene.predict(X_val)
nitrogen_oxides_preds_lr = linear_reg_nitrogen_oxides.predict(X_val)

In [67]:
# Initialize RandomForestRegressor models
rf_carbon_monoxide = RandomForestRegressor(n_estimators=100, random_state=42)
rf_benzene = RandomForestRegressor(n_estimators=100, random_state=42)
rf_nitrogen_oxides = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit RandomForestRegressor models
rf_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])
rf_benzene.fit(X_train, y_train['target_benzene'])
rf_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])

# Predict using RandomForestRegressor models
carbon_monoxide_preds_rf = rf_carbon_monoxide.predict(X_val)
benzene_preds_rf = rf_benzene.predict(X_val)
nitrogen_oxides_preds_rf = rf_nitrogen_oxides.predict(X_val)


In [68]:
# Calculate RMSE for Linear Regression
rmse_carbon_monoxide_lr = np.sqrt(mean_squared_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds_lr))
rmse_benzene_lr = np.sqrt(mean_squared_error(y_val['target_benzene'], benzene_preds_lr))
rmse_nitrogen_oxides_lr = np.sqrt(mean_squared_error(y_val['target_nitrogen_oxides'], nitrogen_oxides_preds_lr))

# Calculate RMSE for Random Forest Regression
rmse_carbon_monoxide_rf = np.sqrt(mean_squared_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds_rf))
rmse_benzene_rf = np.sqrt(mean_squared_error(y_val['target_benzene'], benzene_preds_rf))
rmse_nitrogen_oxides_rf = np.sqrt(mean_squared_error(y_val['target_nitrogen_oxides'], nitrogen_oxides_preds_rf))

# Calculate RMSLE
rmsle_carbon_monoxide_rf = np.sqrt(mean_squared_log_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds_rf))
rmsle_benzene_rf = np.sqrt(mean_squared_log_error(y_val['target_benzene'], benzene_preds_rf))
rmsle_nitrogen_oxides_rf = np.sqrt(mean_squared_log_error(y_val['target_nitrogen_oxides'], nitrogen_oxides_preds_rf))


In [69]:
# Prepare data for tabulate
headers = ['Model', 'RMSE', 'RMSLE']
data = [['Linear Regression (Carbon Monoxide)', rmse_carbon_monoxide_lr, '-'],
        ['Linear Regression (Benzene)', rmse_benzene_lr, '-'],
        ['Linear Regression (Nitrogen Oxides)', rmse_nitrogen_oxides_lr, '-'],
        ['Random Forest Regression (Carbon Monoxide)', rmse_carbon_monoxide_rf, rmsle_carbon_monoxide_rf],
        ['Random Forest Regression (Benzene)', rmse_benzene_rf, rmsle_benzene_rf],
        ['Random Forest Regression (Nitrogen Oxides)', rmse_nitrogen_oxides_rf, rmsle_nitrogen_oxides_rf]]

# Print results using tabulate
print(tabulate(data, headers=headers))

Model                                             RMSE  RMSLE
------------------------------------------  ----------  -------------------
Linear Regression (Carbon Monoxide)           0.616533  -
Linear Regression (Benzene)                   1.83891   -
Linear Regression (Nitrogen Oxides)         113.654     -
Random Forest Regression (Carbon Monoxide)    0.483088  0.1361581831870301
Random Forest Regression (Benzene)            1.30728   0.09393850643737371
Random Forest Regression (Nitrogen Oxides)   79.3434    0.33143521875786836


**Submission File**

In [70]:
carbon_monoxide_test_preds = rf_carbon_monoxide.predict(test_data[features])
benzene_test_preds = rf_benzene.predict(test_data[features])
nitrogen_oxides_test_preds = rf_nitrogen_oxides.predict(test_data[features])

submission = pd.DataFrame({
    'date_time': test_data['date_time'],
    'target_carbon_monoxide': carbon_monoxide_test_preds,
    'target_benzene': benzene_test_preds,
    'target_nitrogen_oxides': nitrogen_oxides_test_preds
})

submission.to_csv('submission.csv', index=False)


# **Gradio**


In [71]:
def airpollution(deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5):
    df = pd.DataFrame.from_dict({'deg_C': [deg_C], 'relative_humidity': [relative_humidity], 'absolute_humidity': [absolute_humidity],'sensor_1':[sensor_1],'sensor_2':[sensor_2],'sensor_3':[sensor_3],'sensor_4':[sensor_4],'sensor_5':[sensor_5]})
    pred = rf_carbon_monoxide.predict(df)[0]
    pred2 = rf_benzene.predict(df)[0]
    pred3 =rf_nitrogen_oxides.predict(df)[0]
    print(pred,pred2,pred3)
    return pred,pred2,pred3

In [72]:
deg_C = gr.inputs.Textbox( label="deg_C")
relative_humidity= gr.inputs.Textbox(label="relative_humidity")
absolute_humidity= gr.inputs.Textbox(label="absolute_humidity")
sensor_1= gr.inputs.Textbox(label="sensor_1")
sensor_2= gr.inputs.Textbox(label="sensor_2")
sensor_3= gr.inputs.Textbox(label="sensor_3")
sensor_4= gr.inputs.Textbox(label="sensor_4")
sensor_5= gr.inputs.Textbox(label="sensor_5")
# CarbonMonoxide= gr.outputs.Textbox(label="CarbonMonoxide")
# Benzene = gr.outputs.Textbox(label="Benzene")
# NitrogenOxide = gr.outputs.Textbox(label="NitrogenOxide")
output = [gr.outputs.Textbox(label="CarbonMonoxide"),gr.outputs.Textbox(label="Benzene"), gr.outputs.Textbox(label="NitrogenOxide")]
gr.Interface(airpollution,[deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5],output,title="Air Pollution Prediction").launch();

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>